In [63]:
import ipyleaflet as L
import numpy as np
from dataloader import TripsLoader
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd

In [64]:
loader = TripsLoader('03-09-2022')

In [65]:
#Dumper statistics
nb_of_dumpers = 0
km_total = 0
quantity_total = 0
duration_total = 0
dumper_dict = {}

for machine_key in tqdm(loader._machines.keys()):
    machine = loader._machines[machine_key]
    if machine.machine_type=='Dumper':
        nb_of_dumpers += 1
        km_total += machine.total_length
        duration_total += machine.total_duration
        dumper_dict[machine.machine_id] = {
            'nb_of_trips': len(machine.trips),
            'total_length': machine.total_length,
            'total_quantity': machine.total_quantity,
            'total_duration': machine.total_duration,
            'nb_stone_trips': len(machine.stone_trips),
            'nb_soil_trips': len(machine.soil_trips),
            'nb_equip_trips': len(machine.equipment_trips),
            'nb_four_trips': len(machine.four_trips)
        }


100%|██████████| 27/27 [00:00<00:00, 50.13it/s]


In [66]:
nb_of_dumpers

5

In [67]:
dumper_pd = pd.DataFrame.from_dict(dumper_dict, orient='index')

In [68]:
dumper_pd

,nb_of_trips,total_length,total_quantity,total_duration,nb_stone_trips,nb_soil_trips,nb_equip_trips,nb_four_trips
41,25,23.630572,750.0,1157.123417,23,2,0,0
37,16,18.843574,520.0,572.050167,14,2,0,0
19,17,14.020311,374.0,246.169200,17,0,0,0
36,7,17.483050,210.0,182.067900,7,0,0,0
27,15,8.938519,600.0,1003.496133,11,3,1,0


In [69]:
all_positions_of_dumpers = []

In [70]:
for machine_key in loader._machines.keys():
    machine = loader._machines[machine_key]
    if machine.machine_type=='Dumper':
        for trip in machine.trips:
            for pos in trip.positions:
                all_positions_of_dumpers.append((pos.lat, pos.lon))

coordinates_array = np.array(all_positions_of_dumpers)

In [71]:
# Create a map centered at the mean of all coordinates, with heatmap
map_center = np.mean(coordinates_array, axis=0)
m = L.Map(center=(map_center[0], map_center[1]), zoom=10)

# Add markers for each cluster center to the map
#for center in cluster_centers:
#    marker = L.Marker(location=(center[0], center[1]))
#    m.add_layer(marker)
heatmap = L.Heatmap(locations=all_positions_of_dumpers,radius=20)
m.add_layer(heatmap)

# Display the map
m

#Can see two clear clusters in the map, maybe three. Can use K-means to find center of them.

Map(center=[59.942684897971745, 10.405920494033346], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [72]:
# Fit the KMeans model with the optimal K value
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
kmeans.fit(coordinates_array)

# Get the coordinates of the cluster centers for the optimal K value
cluster_centers = kmeans.cluster_centers_

In [73]:
# Create a map centered at the mean of all coordinates, with heatmap
map_center = np.mean(coordinates_array, axis=0)
m = L.Map(center=(map_center[0], map_center[1]), zoom=10)

# Add markers for each cluster center to the map
for center in cluster_centers:
    marker = L.Marker(location=(center[0], center[1]))
    m.add_layer(marker)
heatmap = L.Heatmap(locations=all_positions_of_dumpers,radius=20)
m.add_layer(heatmap)

# Display the map
m

Map(center=[59.942684897971745, 10.405920494033346], controls=(ZoomControl(options=['position', 'zoom_in_text'…